In [12]:
import simpy

In [13]:
# env = simpy.Environment()

# def car(env):
#     while True:
#         print('Start parking at %d' % env.now)
#         yield env.timeout(5)

#         print('Start driving at %d' % env.now)
#         yield env.timeout(2)

# env.process(car(env))
# env.run(until=15)

In [14]:
# class Car(object):
#     def __init__(self, env):
#         self.env = env
#         # Start the run process everytime an instance is created.
#         self.action = env.process(self.run())

#     def run(self):
#         while True:
#             print('Start parking and charging at %d' % self.env.now)
#             charge_duration = 5
#             # We yield the process that process() returns
#             # to wait for it to finish
#             yield self.env.process(self.charge(charge_duration))

#             # The charge process has finished and
#             # we can start driving again.
#             print('Start driving at %d' % self.env.now)
#             trip_duration = 2
#             yield self.env.timeout(trip_duration)

#     def charge(self, duration):
#         yield self.env.timeout(duration)

In [15]:
# env = simpy.Environment()
# car = Car(env)
# env.run(until=15)

In [19]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [20]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12
